# Using CirceR

by Anja van Gestel

In [ ]:
## in R:
# remotes::install_github("OHDSI/CirceR")

%load_ext autoreload
%autoreload 2

import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1" # change as needed
from pathlib import Path
import json

from ohdsi import circe

# Introduction

A R-wrapper for the Circe library, allowing OMOP CDM cohort definition expressions to be edited, and converted to JSON or SQL. This package provides convenient wrappers for Circe functions, and includes the necessary Java dependencies.

# JSON input

It's common practice, after the cohort definition stage, to have your cohort definition packed as a JSON file.<br>
For this tutorial we created a JSON file for a "Death cohort", including some concept sets <i>(that are not used in the cohort definition, but that's not a problem)</i>. The JSON file was exported from Atlas.

In [ ]:
cohort_json_filename = r"death_cohort_with_concept_sets.json"
cohort_json = Path(cohort_json_filename).read_text()

# Cohort JSON &rarr; Cohort Expression
First there's the function <code>cohort_expression_from_json()</code> that reads a JSON and deserializes it into a R <code>CohortExpression</code> instance.

In [ ]:
cohort_expression = circe.cohort_expression_from_json(cohort_json)

With the function <code>cohort_print_friendly()</code> you can print this cohort expression:

In [ ]:
print(circe.cohort_print_friendly(cohort_expression))

## Cohort Expression &rarr; Sql

The function <code>createGenerateOptions()</code> creates a generation options object for use in <code>buildCohortQuery()</code>. Which we need to generate the OMOP CDM Sql version of the cohort expression.

In [ ]:
options = circe.create_generate_options()
cohort_sql = circe.build_cohort_query(cohort_expression, options)

# Concept Sets

In [ ]:
with open(cohort_json_filename, 'r') as f:
    cohort_json = json.load(f)
    
concept_sets = cohort_json.get('ConceptSets')

Use <code>concept_set_list_print_friendly()</code> to print all concept set expressions:

In [ ]:
concept_set_list_json = json.dumps(concept_sets)
print(circe.concept_set_list_print_friendly(concept_set_list_json))

Or use the function <code>concept_set_print_friendly()</code> to print individual concept sets expressions:

In [ ]:
single_concept_set = cohort_json.get('ConceptSets')[0]

In [ ]:
concept_set_json = json.dumps(single_concept_set)
print(circe.concept_set_print_friendly(concept_set_json))

## Concept Set Expression from JSON

The cohort definition JSON includes the concept set expressions for the concept sets that were added. However, you can also export a concept set expression JSON from Atlas per individual concept set, that JSON has the same structure as the <code>'expression'</code> key in the cohort definition JSON. To create a <code>ConceptSetExpression</code> instance for a concept set expression use the following:

In [ ]:
single_concept_set_expression = cohort_json.get('ConceptSets')[0].get('expression')
concept_set_expression = circe.concept_set_expression_from_json(single_concept_set_expression)

Note that the cell above now created a concept set expression instance for the first (<code>[0]</code>) concept set, not the second one. It only works PER concept set.

## Concept Set Expression &rarr; Sql

In [ ]:
circe.build_concept_set_query(single_concept_set_expression)